# Analyse a sample of FUAs

This notebook produces face polygons and computes shape metrics for each of them.

In [1]:
import warnings

import geopandas
import dask_geopandas
import numpy
import pygeos
import esda.shape as shape

from tqdm import tqdm

In [2]:
sample = geopandas.read_parquet("../data/sample.parquet")

## Measure shape characteristics

Polygonize the network to get polygons fully enclosed by street network geometry (face polygons) and measure their shape characteristics.

In [3]:
# Filter warnings about GeoParquet implementation.
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

# Loop over unique FUA IDs
for fua_id in tqdm(sample.eFUA_ID, total=len(sample)):
    # Read stret network
    roads = geopandas.read_parquet(f"../data/{int(fua_id)}/roads_osm.parquet")

    # remove service roads
    roads = roads[roads.highway != "service"]

    # Polygonize street network
    polygons = pygeos.polygonize(roads.geometry.array.data)
    
    # Store geometries as a GeoDataFrame
    polygons = geopandas.GeoDataFrame(
        geometry=geopandas.GeoSeries(
            [polygons], crs=roads.crs
        ).explode(ignore_index=True)
    )
    
    # Ensure all polygons are valid
    if not polygons.is_valid.all():
        polygons = geopandas.GeoDataFrame(
        geometry=geopandas.GeoSeries(
            pygeos.make_valid(polygons.geometry.values.data), crs=roads.crs
        ).explode(ignore_index=True)
    )
    
    # Ensure that all geometries are polygons
    if not (polygons.geom_type == "Polygon").all():
        polygons = polygons[polygons.geom_type == "Polygon"].reset_index(drop=True)

    area = polygons.area

    # measure (circular) compactness
    polygons["circular_compactness"] = shape.minimum_bounding_circle_ratio(polygons)
    polygons["circular_compactness_index"] = polygons["circular_compactness"] * area
    
    # isoperimetric_quotient
    polygons["isoperimetric_quotient"] = shape.isoperimetric_quotient(polygons)
    polygons["isoperimetric_quotient_index"] = polygons["isoperimetric_quotient"] * area


    # isoareal_quotient
    polygons["isoareal_quotient"] = shape.isoareal_quotient(polygons)
    polygons["isoareal_quotient_index"] = polygons["isoareal_quotient"] * area

    # radii_ratio
    polygons["radii_ratio"] = shape.radii_ratio(polygons)
    polygons["radii_ratio_index"] = polygons["radii_ratio"] * area

    # diameter_ratio
    polygons["diameter_ratio"] = shape.diameter_ratio(polygons)
    polygons["diameter_ratio_index"] = polygons["diameter_ratio"] * area

    # fractal_dimension
    polygons["fractal_dimension"] = shape.fractal_dimension(polygons, support="square")
    polygons["fractal_dimension_index"] = polygons["fractal_dimension"] * area

    # save polygons to a partitioned GeoParquet
    polygons = dask_geopandas.from_geopandas(polygons, npartitions=10)
    polygons.to_parquet(f"../data/{int(fua_id)}/polygons/")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [07:01<00:00,  3.22s/it]
